# Automl feature selector

## Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pwd

/content


In [3]:
%cd "drive/MyDrive/22-23/DS 301 Advanced Data Science/DS 301 Final Project"

/content/drive/.shortcut-targets-by-id/1L_EikC2ryU5sSu8xLqkBA2WHnRs7BMt-/DS 301 Final Project


In [4]:
!pwd

/content/drive/.shortcut-targets-by-id/1L_EikC2ryU5sSu8xLqkBA2WHnRs7BMt-/DS 301 Final Project


## Import

In [5]:
# importing the libraries

import pandas as pd
import numpy as np
import copy

# for visualization

import plotly.express as px
import matplotlib.pyplot as plt


In [6]:
# models

import statsmodels.api as sm
import sklearn.model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import xgboost

In [7]:
# dict used to convert state to abbrev

us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}


## Reading data/helper functions

In [8]:
# Reading the training data

data = pd.read_csv('state_data_2019_processed_training.csv')

states = data.sitecode.unique()

In [9]:
# Creating the training/testing sets

state_data_t = []

state_data_test = []

seed = 100

for state in states:

    state_data = data[data.sitecode == state]
    X_s = state_data.iloc[:,:-1]
    y_s = state_data.iloc[:,-1]
    X_tr, X_val, y_tr, y_val = sklearn.model_selection.train_test_split(
        X_s, y_s, test_size=0.4, random_state=seed, stratify=y_s)
    X_val, X_te, y_val, y_te = sklearn.model_selection.train_test_split(
        X_val, y_val, test_size=0.5, random_state=seed, stratify=y_val)
    state_data_t.append([X_tr, X_val, y_tr, y_val])
    state_data_test.append((X_te, y_te))

X_train, X_val, y_train, y_val = state_data_t[0]

for i in range(1, len(state_data_t)):
    X_t,X_v,y_t,y_v = state_data_t[i]
    X_train = pd.concat([X_train, X_t])
    y_train = pd.concat([y_train, y_t])
    X_val = pd.concat([X_val, X_v])
    y_val = pd.concat([y_val, y_v])

In [10]:
# functions for visualizing the results

def map(model, race = 'all', exclude = None, data = state_data_t):

    auc_all = pd.DataFrame({'state': [us_state_to_abbrev[s] for s in states], 'auc': np.zeros(len(states))})

    for i, state in enumerate(states):
        _, X_test, _, y_test = data[i]
        if(race != 'all'):
            if(race == 'black'):
                index = X_test['race4_Black or African American'] == 1
            if(race == 'hispanic'):
                index = X_test['race4_Hispanic/Latino'] == 1
            X_test = X_test.loc[index]
            y_test = y_test.loc[index]
        if(exclude != None):
            X_test = X_test.drop(columns = exclude)
        y_pred = model.predict_proba(X_test.iloc[:,4:])
        fpr, tpr,_ = metrics.roc_curve(y_test, y_pred[:,1])
        auc = metrics.auc(fpr,tpr)
        auc_all.iloc[i,1] = auc

    fig = px.choropleth(auc_all,
                        locations='state', 
                        locationmode="USA-states", 
                        scope="usa",
                        color='auc',
                        color_continuous_scale="Viridis_r",
                        range_color = [0.6,1] 
                        )
    return fig, auc_all

def count_race(data = state_data_t):
    race = pd.DataFrame({
        'others': np.zeros(len(states)),
        'black': np.zeros(len(states)),
        'hispanic': np.zeros(len(states)),
        'white': np.zeros(len(states)),
        'missing': np.zeros(len(states))
    }, index = states)
    for i, state in enumerate(states):
        _, X_test, _, _ = data[i]
        race.iloc[i,0] = X_test['race4_All other races'].sum()
        race.iloc[i,1] = X_test['race4_Black or African American'].sum()
        race.iloc[i,2] = X_test['race4_Hispanic/Latino'].sum()
        race.iloc[i,3] = X_test['race4_White'].sum()
        race.iloc[i,4] = X_test['race4_All other races'].count() - \
            race.iloc[i,0] - race.iloc[i,1] -race.iloc[i,2] -race.iloc[i,3]
    
    return race

## AutoML

### Install

In [25]:
!pip install requests
!pip install tabulate
!pip install future

!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
     |████████████████████████████████| 177.5 MB 38 kB/s 
  Created wheel for h2o: filename=h2o-3.38.0.3-py2.py3-none-any.whl size=177541401 sha256=a099ec496d01ff0ad6fd9a013bb50c81a1ab1775ce8b861a8e7475c905cd822d
  Stored in directory: /root/.cache/pip/wheels/16/f8/f4/69e1ff6a0d1cb61bdbc0d9888ee8437a1acf1eb1c6ffb8be20
Successfully built h2o


### Running AutoML

In [26]:
import h2o

h2o.init()

predictors = X_train.columns[4:].to_list()
response = 'qn28'
train = pd.concat([X_train.iloc[:,4:], y_train.astype('category')], axis = 1)
valid = pd.concat([X_val.iloc[:,4:], y_val.astype('category')], axis = 1)
train = h2o.H2OFrame(train)
valid = h2o.H2OFrame(valid)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.17" 2022-10-18; OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.8/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp8pekapif
  JVM stdout: /tmp/tmp8pekapif/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp8pekapif/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.3
H2O_cluster_version_age:,25 days
H2O_cluster_name:,H2O_from_python_unknownUser_go1z46
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [27]:
train['qn28'] = train['qn28'].asfactor()
valid['qn28'] = valid['qn28'].asfactor()

In [29]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()

aml = H2OAutoML(max_models=20, seed=1,
              nfolds = 0)
aml.train(x=predictors, y=response,
              training_frame=train,
              validation_frame=valid
          )

#24 min

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,24 mins 01 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.3
H2O_cluster_version_age:,25 days
H2O_cluster_name:,H2O_from_python_unknownUser_go1z46
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.026 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


AutoML progress: |
19:35:24.939: _train param, Dropping bad and constant columns: [stheight_1.19]

██
19:36:15.533: _train param, Dropping bad and constant columns: [stheight_1.19]

███████████████████
19:37:05.264: _train param, Dropping bad and constant columns: [stheight_1.19]
19:38:59.294: _train param, Dropping bad and constant columns: [stheight_1.19]

██
19:39:22.370: _train param, Dropping bad and constant columns: [stheight_1.19]

███████████
19:42:02.531: _train param, Dropping bad and constant columns: [stheight_1.19]
19:43:01.824: _train param, Dropping bad and constant columns: [stheight_1.19]

█████
19:44:11.584: _train param, Dropping bad and constant columns: [stheight_1.19]

██████
19:45:39.95: _train param, Dropping bad and constant columns: [stheight_1.19]

█
19:45:56.256: _train param, Dropping bad and constant columns: [stheight_1.19]


19:46:10.417: _train param, Dropping bad and constant columns: [stheight_1.19]

███
19:47:05.386: _train param, Dropping bad and c

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_5_AutoML_2_20221218_193524


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    60                 60                          50021                  6            6            6             43            64            61.7333

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.05428198187128812
RMSE: 0.23298493915119947
LogLoss: 0.19331774770565868
Mean Per-Class Error: 0.24118507567753464
AUC: 0.8986362317707008
AUCPR: 0.6208776759052915
Gini: 0.7972724635414017

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.25571377364619513
       0      1     Error    Rate
-----  -----  ----  -------  ----------------
0      65568  3157  0.0459   (3157.0/68725.0)
1      3069   3963  0.4364   (3069.0/7032.0)
Total  68637  7120  0.0822   (6226.0/75757.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.255714     0.560062  199
max f2                       0.127092     0.619195  270
max f0point5                 0.423661     0.639416  136
max accuracy                 0.423661     0.932019  136
max precision                0.992789     1         0
max recall                   0.00877387   1         397
max specificity              0.992789     1         0
max absolute_mcc             0.327402     0.521056  170
max min_per_class_accuracy   0.100572     0.812429  289
max mean_per_class_accuracy  0.100572     0.812879  289
max tns                      0.992789     68725     0
max fns                      0.992789     7031      0
max fps                      0.00779488   68725     399
max tps                      0.00877387   7032      397
max tnr                      0.992789     1         0
max fnr                      0.992789     0.999858  0
max fpr                      0.00779488   1         399
max tpr                      0.00877387   1         397

Gains/Lift Table: Avg response rate:  9.28 %, avg score:  9.29 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100057                   0.798988           10.7163    10.7163            0.994723         0.872893   0.994723                    0.872893            0.107224        0.107224                   971.633   971.633            0.107166
2        0.0200114                   0.662247           9.53668    10.1265            0.885224         0.727736   0.939974                    0.800315            0.0954209       0.202645                   853.668   912.65             0.201321
3        0.0300038                   0.549441           8.04075    9.43186            0.746367         0.603237   0.875495                    0.73468             0.080347        0.282992                   704.075   843.186            0.278874
4        0.0400095                   0.47088            6.4099     8.67612            0.594987         0.508536   0.805345                    0.678125            0.0641354       0.347127                   540.99    767.612            0.338542
5        0.050002                    0.409945           5.46486    8.03438            0.507266         0.440321   0.745776      

In [30]:
aml.leader.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'GBM_5_AutoML_2_20221218_193524',
   'type': 'Key<Model>',
   'URL': '/3/Models/GBM_5_AutoML_2_20221218_193524'},
  'input': None},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'AutoML_2_20221218_193524_training_py_1_sid_96fe',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/AutoML_2_20221218_193524_training_py_1_sid_96fe'},
  'input': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'AutoML_2_20221218_193524_training_py_1_sid_96fe',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/AutoML_2_20221218_193524_training_py_1_sid_96fe'}},
 'validation_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type':

In [31]:
model_path = h2o.save_model(model=aml.leader, path="/tmp/mymodel", force=True)
print(model_path)

/tmp/mymodel/GBM_5_AutoML_2_20221218_193524


### Examining final model

In [32]:
X_test, y_test = state_data_test[0]

for i in range(1,len(state_data_test)):
  X_t,  y_t = state_data_test[i]
  X_test = pd.concat([X_test, X_t])
  y_test = pd.concat([y_test, y_t])

In [33]:
X_test = X_test.iloc[:,4:]

In [ ]:
X_test = h2o.H2OFrame(X_test)

In [35]:
y_pred = aml.leader.predict(X_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [42]:
y_pred['p1']

p1
0.344594
0.123808
0.081307
0.142592
0.0764039
0.172917
0.434619
0.328944
0.117184
0.406527


In [44]:
fpr,tpr, _ = metrics.roc_curve(y_test, y_pred['p1'].as_data_frame())

In [45]:
metrics.roc_auc_score(y_test, y_pred['p1'].as_data_frame())

0.8640388868734653

## Feature Engineering

In [11]:
!pip install autofeat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 269 kB 7.1 MB/s 


In [23]:
from autofeat import FeatureSelector, AutoFeatRegressor

X, _, y, _ = sklearn.model_selection.train_test_split(X_train.iloc[:,4:],y_train,train_size = 0.2, stratify=y_train)

nunique = X.nunique()
cols_to_drop = nunique[nunique == 1].index
X = X.drop(columns = cols_to_drop)

In [24]:
fsel = FeatureSelector(verbose=1, problem_type="classification")
new_X = fsel.fit_transform(X, y)

[featsel] Scaling data...done.
[featsel] Feature selection run 1/5


KeyboardInterrupt: ignored

In [ ]:
print(X.columns)
print(new_X.columns)